<a href="https://colab.research.google.com/github/lleiva25/San_Francisco_Survey_Analysis/blob/main/San_Francisco_Survey_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>San Francisco: Public Space, Film Sight, & 311 Corelation </h1>

<h2>Data Cleanup Draft</h2>

In [ ]:
pip install sodapy

In [ ]:
#Import Dependencies
import pandas as pd
import numpy as np
import json
import requests
from sodapy import Socrata

In [ ]:
#File paths
#film_file_path = 'Resources/Film_Locations_in_San_Francisco_20240116.csv'
film_file_path = '/Film_Locations_in_San_Francisco_20240116.csv'



In [ ]:
#Create Dataframes from resource files
film_df = pd.read_csv(film_file_path)
film_df.head()

,Title,Release Year,Locations,Fun Facts,Production Company,Distributor,Director,Writer,Actor 1,Actor 2,Actor 3,SF Find Neighborhoods,Analysis Neighborhoods,Current Supervisor Districts
0,Experiment in Terror,1962,The Sea Captain's Chest (Fisherman's Wharf),NaN,Columbia Pictures Corporation,Columbia Pictures,Blake Edwards,The Gordons,Glenn Ford,Lee Remick,Stefanie Powers,99.0,23.0,3.0
1,Experiment in Terror,1962,100 St. Germain Avenue,NaN,Columbia Pictures Corporation,Columbia Pictures,Blake Edwards,The Gordons,Glenn Ford,Lee Remick,Stefanie Powers,47.0,38.0,8.0
2,Chan is Missing,1982,"Li Po (916 Grant Avenue at Washington, Chinatown)",NaN,New Yorker Films,New Yorker Films,Wayne Wang,Wayne Wang,Wood Moy,Marc Hayashi,Lauren Chew,104.0,6.0,3.0
3,A View to a Kill,1985,Taylor and Jefferson Streets (Fisherman's Wharf),NaN,Metro-Goldwyn Mayer,MGM/UA Entertainment Company,John Glen,Richard Maibaum,Roger Moore,Christopher Walken,Tanya Roberts,99.0,23.0,3.0
4,The Californians,2005,NaN,NaN,Parker Film Company,Fabrication Films,Jonathan Parker,Jonathan Parker & Catherine DiNapoli,Noah Wyle,NaN,NaN,21.0,36.0,10.0


In [ ]:
#Use this for nested JSONs
#Get data from API via request
response_public = requests.get(geojson_Public_Spaces).json()

#Finding the length of the json request
end_range = len(response_public['features'])

#Create list to obtain data from loop
public_space_dict = []

#Loop through json by length
for i in range(end_range):
    public_space_dict.append({'Name':response_public['features'][i]['properties']['name'],
                              'Hours':response_public['features'][i]['properties']['hours_type'],
                              'Year Opened':response_public['features'][i]['properties']['year'],
                              'Food Onsite':response_public['features'][i]['properties']['food_service'],
                              'Restrooms':response_public['features'][i]['properties']['restrooms'],
                              'Address':response_public['features'][i]['properties']['popos_address'],
                              'Capacity':response_public['features'][i]['properties']['seating_no'],
                              'Landscaping':response_public['features'][i]['properties']['landscaping'],
                              'Amenities':response_public['features'][i]['properties']['amenities']})

#Put the JSON data into a pandas dataframe
public_space_df = pd.DataFrame(public_space_dict)
public_space_df.head()

,Name,Hours,Year Opened,Food Onsite,Restrooms,Address,Capacity,Landscaping,Amenities
0,555 Mission St,Open At All Times,2008,Yes,Yes,555 Mission St,100 linear seats and benches.,"9 trees, permeable gravel, bushes",None
1,Foundry Square NE,Open At All Times,2008,Yes,No,400 Howard St,"20 linear seats, 24 movable",8 trees,None
2,Foundry Square SE,Open Business Hours,2005,Yes,No,405 Howard St,"44 movable chairs, 45 linear seats on planter ...",8 trees,None
3,Foundry Square NW,Open At All Times,2003,Yes,No,500 Howard St,"44 linear seating on ledges, 21 seating on pla...","8 trees, palm plant planters,",None
4,235 Second St,Open Business Hours,2002,Yes,No,235 Second St,42 chairs and 15 tables,Bamboo and street trees protect space from sid...,None


In [ ]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.sfgov.org", None)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("vw6y-z8j6", offset=1000) #limit=3268164)

# Convert to pandas DataFrame
results_311_df = pd.DataFrame.from_records(results)

len(results_311_df)

1000

In [ ]:
dir(client)

['DEFAULT_LIMIT',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_perform_request',
 '_perform_update',
 'close',
 'create',
 'create_non_data_file',
 'datasets',
 'delete',
 'domain',
 'download_attachments',
 'get',
 'get_all',
 'get_metadata',
 'publish',
 'replace',
 'replace_non_data_file',
 'session',
 'set_permission',
 'timeout',
 'update_metadata',
 'upsert',
 'uri_prefix']

,service_request_id,requested_datetime,closed_date,updated_datetime,status_description,status_notes,agency_responsible,service_name,service_subtype,service_details,...,:@computed_region_2dwj_jsy4,:@computed_region_9jxd_iqea,media_url,:@computed_region_9dfj_4gjx,:@computed_region_4isq_27mq,:@computed_region_pigm_ib2e,:@computed_region_n4xg_c4py,:@computed_region_viu7_rrfi,:@computed_region_6ezc_tdp2,:@computed_region_h4ep_8xdi
0,1000040,2011-11-08T10:10:55.000,2011-11-11T06:07:07.000,2011-11-11T06:07:07.000,Closed,See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000203,2011-11-08T12:12:49.000,2011-11-08T13:07:08.000,2011-11-08T13:07:08.000,Closed,See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000266,2011-11-08T13:28:24.000,2011-11-09T06:07:22.000,2011-11-09T06:07:22.000,Closed,See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000532,2011-11-08T18:46:38.000,2011-11-10T07:03:29.000,2011-11-10T07:03:29.000,Closed,Case Transferred - PSR 399382,SSP - MTA Feedback Queue,MUNI Feedback,MUNI - Services_Miscellaneous,801_NextMuni_Technology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000605,2011-11-08T23:58:51.000,2011-11-22T08:18:15.000,2011-11-22T08:18:15.000,Closed,Case Resolved,Elections - G,General Request - ELECTIONS,complaint,elections - complaint,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#API
api_311 = 'https://data.sfgov.org/resource/vw6y-z8j6.json?$query=SELECT%0A%20%20%60service_request_id%60%2C%0A%20%20%60requested_datetime%60%2C%0A%20%20%60closed_date%60%2C%0A%20%20%60updated_datetime%60%2C%0A%20%20%60status_description%60%2C%0A%20%20%60status_notes%60%2C%0A%20%20%60agency_responsible%60%2C%0A%20%20%60service_name%60%2C%0A%20%20%60service_subtype%60%2C%0A%20%20%60service_details%60%2C%0A%20%20%60address%60%2C%0A%20%20%60street%60%2C%0A%20%20%60supervisor_district%60%2C%0A%20%20%60neighborhoods_sffind_boundaries%60%2C%0A%20%20%60police_district%60%2C%0A%20%20%60lat%60%2C%0A%20%20%60long%60%2C%0A%20%20%60point%60%2C%0A%20%20%60point_geom%60%2C%0A%20%20%60source%60%2C%0A%20%20%60media_url%60%2C%0A%20%20%60bos_2012%60%2C%0A%20%20%60data_as_of%60%2C%0A%20%20%60data_loaded_at%60%2C%0A%20%20%60%3A%40computed_region_6qbp_sg9q%60%2C%0A%20%20%60%3A%40computed_region_qgnn_b9vv%60%2C%0A%20%20%60%3A%40computed_region_26cr_cadq%60%2C%0A%20%20%60%3A%40computed_region_ajp5_b2md%60%2C%0A%20%20%60%3A%40computed_region_rxqg_mtj9%60%2C%0A%20%20%60%3A%40computed_region_yftq_j783%60%2C%0A%20%20%60%3A%40computed_region_jx4q_fizf%60%2C%0A%20%20%60%3A%40computed_region_bh8s_q3mv%60%2C%0A%20%20%60%3A%40computed_region_p5aj_wyqh%60%2C%0A%20%20%60%3A%40computed_region_fyvs_ahh9%60%2C%0A%20%20%60%3A%40computed_region_f58d_8dbm%60%2C%0A%20%20%60%3A%40computed_region_9dfj_4gjx%60%2C%0A%20%20%60%3A%40computed_region_vtsz_7cme%60%2C%0A%20%20%60%3A%40computed_region_n4xg_c4py%60%2C%0A%20%20%60%3A%40computed_region_sruu_94in%60%2C%0A%20%20%60%3A%40computed_region_4isq_27mq%60%2C%0A%20%20%60%3A%40computed_region_viu7_rrfi%60%2C%0A%20%20%60%3A%40computed_region_fcz8_est8%60%2C%0A%20%20%60%3A%40computed_region_pigm_ib2e%60%2C%0A%20%20%60%3A%40computed_region_9jxd_iqea%60%2C%0A%20%20%60%3A%40computed_region_6ezc_tdp2%60%2C%0A%20%20%60%3A%40computed_region_6pnf_4xz7%60%2C%0A%20%20%60%3A%40computed_region_h4ep_8xdi%60%2C%0A%20%20%60%3A%40computed_region_nqbw_i6c3%60%2C%0A%20%20%60%3A%40computed_region_2dwj_jsy4%60%2C%0A%20%20%60%3A%40computed_region_y6ts_4iup%60%2C%0A%20%20%60%3A%40computed_region_jwn9_ihcz%60%0AWHERE%0A%20%20%60requested_datetime%60%0A%20%20%20%20BETWEEN%20%222008-01-01T00%3A00%3A00%22%20%3A%3A%20floating_timestamp%0A%20%20%20%20AND%20%222018-12-31T23%3A45%3A00%22%20%3A%3A%20floating_timestamp%0AORDER%20BY%0A%20%20%60requested_datetime%60%20ASC%20NULL%20LAST%2C%0A%20%20%60closed_date%60%20DESC%20NULL%20LAST'
geojson_Public_Spaces = 'https://data.sfgov.org/resource/65ik-7wqd.geojson'
#crime_data =

In [ ]:
#Parsing through large database via API
#Get access to URL to obtain API data
offset = 1000
limit = 2000000
#&$limit={limit}&$offset={offset}

#Get response from API with offsets & limits
response_crime = requests.get(f'https://data.sfgov.org/resource/tmnf-yvry.json?$query=SELECT%0A%20%20%60pdid%60%2C%0A%20%20%60incidntnum%60%2C%0A%20%20%60incident_code%60%2C%0A%20%20%60category%60%2C%0A%20%20%60descript%60%2C%0A%20%20%60dayofweek%60%2C%0A%20%20%60date%60%2C%0A%20%20%60time%60%2C%0A%20%20%60pddistrict%60%2C%0A%20%20%60resolution%60%2C%0A%20%20%60address%60%2C%0A%20%20%60x%60%2C%0A%20%20%60y%60%2C%0A%20%20%60location%60%2C%0A%20%20%60%3A%40computed_region_6qbp_sg9q%60%2C%0A%20%20%60%3A%40computed_region_qgnn_b9vv%60%2C%0A%20%20%60%3A%40computed_region_26cr_cadq%60%2C%0A%20%20%60%3A%40computed_region_ajp5_b2md%60%2C%0A%20%20%60%3A%40computed_region_yftq_j783%60%2C%0A%20%20%60%3A%40computed_region_p5aj_wyqh%60%2C%0A%20%20%60%3A%40computed_region_rxqg_mtj9%60%2C%0A%20%20%60%3A%40computed_region_bh8s_q3mv%60%2C%0A%20%20%60%3A%40computed_region_fyvs_ahh9%60%2C%0A%20%20%60%3A%40computed_region_9dfj_4gjx%60%2C%0A%20%20%60%3A%40computed_region_n4xg_c4py%60%2C%0A%20%20%60%3A%40computed_region_4isq_27mq%60%2C%0A%20%20%60%3A%40computed_region_fcz8_est8%60%2C%0A%20%20%60%3A%40computed_region_pigm_ib2e%60%2C%0A%20%20%60%3A%40computed_region_9jxd_iqea%60%2C%0A%20%20%60%3A%40computed_region_6pnf_4xz7%60%2C%0A%20%20%60%3A%40computed_region_6ezc_tdp2%60%2C%0A%20%20%60%3A%40computed_region_h4ep_8xdi%60%2C%0A%20%20%60%3A%40computed_region_nqbw_i6c3%60%2C%0A%20%20%60%3A%40computed_region_2dwj_jsy4%60%2C%0A%20%20%60%3A%40computed_region_jwn9_ihcz%60%0AWHERE%0A%20%20(%60date%60%20%3E%20%222008-01-01T00%3A00%3A00%22%20%3A%3A%20floating_timestamp)%0A%20%20AND%20(%60date%60%20%3C%20%222018-12-31T23%3A45%3A00%22%20%3A%3A%20floating_timestamp)').json()

#Convert to Dataframe
df_crime = pd.DataFrame(response_crime)


In [ ]:
#Split date to year, month and date/time
df_crime[['year','month','day_time']] = df_crime['date'].str.split('-', expand=True)

#Delete unnecessary columns
del df_crime['date']
del df_crime['pdid']
del df_crime['incidntnum']
del df_crime['incident_code']

#List of columns
del_col = [':@computed_region_6qbp_sg9q',
       ':@computed_region_qgnn_b9vv', ':@computed_region_26cr_cadq',
       ':@computed_region_ajp5_b2md', ':@computed_region_yftq_j783',
       ':@computed_region_p5aj_wyqh', ':@computed_region_rxqg_mtj9',
       ':@computed_region_bh8s_q3mv', ':@computed_region_fyvs_ahh9',
       ':@computed_region_9jxd_iqea', ':@computed_region_6pnf_4xz7',
       ':@computed_region_jwn9_ihcz', ':@computed_region_4isq_27mq',
       ':@computed_region_9dfj_4gjx', ':@computed_region_n4xg_c4py',
       ':@computed_region_pigm_ib2e', ':@computed_region_6ezc_tdp2',
       ':@computed_region_h4ep_8xdi', ':@computed_region_2dwj_jsy4',
       ':@computed_region_fcz8_est8', ':@computed_region_nqbw_i6c3',
       'location']

#Loop for deleting unnecessary columns
for col in del_col:
    del df_crime[col]

#View dataframe
df_crime.head()

,category,descript,dayofweek,time,pddistrict,resolution,address,x,y,year,month,day_time
0,LARCENY/THEFT,PETTY THEFT FROM A BUILDING,Wednesday,16:53,NORTHERN,NONE,1200 Block of VANNESS AV,-122.421441956086,37.7868516471087,2008,01,02T00:00:00.000
1,MISSING PERSON,FOUND PERSON,Wednesday,08:00,BAYVIEW,NONE,1400 Block of PHELPS ST,-122.394438859914,37.7364438996732,2008,01,02T00:00:00.000
2,NON-CRIMINAL,LOST PROPERTY,Wednesday,20:30,SOUTHERN,NONE,MARKET ST / POWELL ST,-122.407545855931,37.784401179486,2008,01,02T00:00:00.000
3,FRAUD,"FORGERY, CREDIT CARD",Wednesday,19:15,TENDERLOIN,NONE,0 Block of POWELL ST,-122.407845374111,37.7850943210103,2008,01,02T00:00:00.000
4,LARCENY/THEFT,PETTY THEFT FROM A BUILDING,Wednesday,13:30,TENDERLOIN,NONE,0 Block of JONES ST,-122.412192555721,37.7816045741252,2008,01,02T00:00:00.000


In [ ]:
#Get access to URL to obtain API data
response_311 = requests.get(f'{api_311}').json()

 #Convert to Dataframe
df_311 = pd.DataFrame(response_311)

df_311.head()

,service_request_id,requested_datetime,closed_date,updated_datetime,status_description,status_notes,agency_responsible,service_name,service_subtype,service_details,...,:@computed_region_jwn9_ihcz,:@computed_region_9jxd_iqea,:@computed_region_n4xg_c4py,:@computed_region_fcz8_est8,:@computed_region_6ezc_tdp2,:@computed_region_h4ep_8xdi,:@computed_region_nqbw_i6c3,:@computed_region_2dwj_jsy4,:@computed_region_viu7_rrfi,:@computed_region_y6ts_4iup
0,240215,2008-07-01T00:13:13.000,2008-07-02T16:05:13.000,2008-07-02T16:05:13.000,Closed,- See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,...,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,240216,2008-07-01T00:26:34.000,2008-07-01T06:10:23.000,2008-07-01T06:10:23.000,Closed,- See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,Bulky Items,Mattress,...,109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,240220,2008-07-01T01:56:58.000,2008-07-01T06:10:23.000,2008-07-01T06:10:23.000,Closed,- See Notes tab for more details,DPW Ops Queue,Tree Maintenance,Trees - Damaged_Tree,Hanging_limb,...,109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,240221,2008-07-01T02:07:11.000,2008-07-08T10:03:41.000,2008-07-08T10:03:41.000,Closed,- DPT Abandoned Vehicles - Gone on Arrival #391,DPT Abandoned Vehicles Work Queue,Abandoned Vehicle,Abandoned Vehicle - Car4door,DARK BLUE - FORD - CROWN VICTORIA - 3WBG858,...,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,240222,2008-07-01T03:03:08.000,2008-07-01T06:10:23.000,2008-07-01T06:10:23.000,Closed,- See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,...,61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

#Replace df with a a new df with the correct columns
df_311_new = pd.DataFrame({
    #'service_request_id': [item['service_request_id'] for item in response_311],
    'requested_datetime': [item['requested_datetime'] for item in response_311],
    #'close_date': [item['closed_date'] for item in response_311],
    'status_description': [item['status_description'] for item in response_311],
    #'status_notes': [item['status_notes'] for item in response_311],
    'agency_responsible': [item['agency_responsible'] for item in response_311],
    'service_name': [item['service_name'] for item in response_311],
    'service_subtype': [item['service_subtype'] for item in response_311],
    #'closed_date1': [item['closed_date'] for item in response_311],
    'address': [item['address'] for item in response_311],
    'lat': [item['lat'] for item in response_311],
    'long': [item['long'] for item in response_311]
})

#Split the date into Year, Month and Date/Time
df_311_new[['year','month','day_time']] = df_311_new['requested_datetime'].str.split('-', expand=True)

#Delete the Unneeded Column
del df_311_new['requested_datetime']

#View dataframe
df_311_new.head()

,status_description,agency_responsible,service_name,service_subtype,address,lat,long,year,month,day_time
0,Closed,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,"408 UNION ST, SAN FRANCISCO, CA, 94133",37.800987243652,-122.406204223633,2008,07,01T00:13:13.000
1,Closed,DPW Ops Queue,Street and Sidewalk Cleaning,Bulky Items,Intersection of 18TH AVE and JUDAH ST,37.761745452881,-122.475921630859,2008,07,01T00:26:34.000
2,Closed,DPW Ops Queue,Tree Maintenance,Trees - Damaged_Tree,Intersection of 2ND AVE and IRVING ST,37.764366149902,-122.45881652832,2008,07,01T01:56:58.000
3,Closed,DPT Abandoned Vehicles Work Queue,Abandoned Vehicle,Abandoned Vehicle - Car4door,"1271 EDDY ST, SAN FRANCISCO, CA, 94115",37.781799316406,-122.428535461426,2008,07,01T02:07:11.000
4,Closed,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Intersection of BROOK ST and MISSION ST,37.740642547607,-122.42308807373,2008,07,01T03:03:08.000


Comparing Years in Dataset

In [ ]:
film_df['Release Year'].unique()

array([1962, 1982, 1985, 2005, 2010, 2011, 2017, 2021, 2022, 1996, 2013,
       1954, 2015, 2018, 2014, 2016, 2006, 2020, 2019, 1983, 1968, 2001,
       1958, 2023, 1991, 1979, 1995, 1974, 1993, 1989, 1976, 1997, 1987,
       1969, 1994, 1998, 2000, 1978, 1950, 1988, 2012, 1992, 1973, 2002,
       1999, 1990, 2003, 1980, 2004, 1971, 1986, 1972, 1948, 1936, 1947,
       1938, 1924, 1984, 1967, 1981, 2008, 1927, 1941, 1961, 1951, 1956,
       1959, 1955, 1935, 1943, 1952, 1964, 1957, 1915, 1942, 1977, 1960,
       2007, 1970, 1963])

In [ ]:
public_space_df['Year Opened'].unique()

array(['2008', '2005', '2003', '2002', '2001', '1968', '2000', '1990',
       '1988', '2013', '1984', 'existed before, updates conditioned 2006',
       '2006', '1986', '1985', '1982', '1989', '1973', '1976', '1983',
       '2019', '1981', '1980', '1979', '1974', '1970', '1972', '1969',
       '1965', '1960', '1959', '2016', '2015', 'Under construction',
       '2011'], dtype=object)

In [ ]:
df_311_new['year'].unique()

array(['2008'], dtype=object)

In [ ]:
df_crime['year'].unique()

array(['2008'], dtype=object)

<h1>Mapping 311 Locations</h1>

In [ ]:
!pip install geopandas
!pip install geodatasets

In [ ]:
#Import dependences
import geopandas as gpd
import geodatasets
import matplotlib.pyplot as plt


In [ ]:
#Viewing available location datasets
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

cities = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))

In [ ]:
world.head()

In [ ]:
#Verifing dataset has San Francisco
cities[cities['name']=='San Francisco']

In [ ]:
cities.

In [ ]:
sf = gpd.read_file(geodatasets.get_path('geoda.malaria'))